In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

In [ ]:
DATA_DIR = '/content/drive/MyDrive/투빅스 컨퍼런스/Data/'

steam_df = pd.read_csv('/content/drive/MyDrive/재테크/Steam/Data/steam_reviews_v2_clean.csv')

In [ ]:
steam_df.head()

,content,recommended,user_id,game_id,timestamp,play_time_minute,review_time_minute,game_content_link,review_helpful_count
0,"Very good game, within 6 hours from launch, th...",Recommended,atgbui,768520,2021-06-20,3042.0,3042.0,https://store.steampowered.com/app/768520,5
1,I cannot recommend this game. It's been 8 year...,Not Recommended,atgbui,251570,2021-06-14,40440.0,40440.0,https://store.steampowered.com/app/251570,48
2,Game is currently busted for Co-Op. Despite th...,Not Recommended,atgbui,671510,2021-06-12,1734.0,906.0,https://store.steampowered.com/app/671510,3
3,Unfortunately dead game. https://steamcharts.c...,Not Recommended,atgbui,613590,2021-06-12,48.0,48.0,https://store.steampowered.com/app/613590,0
4,It just felt too generic- why would I play thi...,Not Recommended,atgbui,374280,2021-05-20,540.0,540.0,https://store.steampowered.com/app/374280,1


In [ ]:
steam_df['game_id'].nunique()

24085

# 데이터 크롤링

In [ ]:
df = []
game_id_li = steam_df['game_id'].unique()
except_df = []

for game_id in tqdm_notebook(game_id_li):

  url = f'https://store.steampowered.com/app/{game_id}'
  req = requests.get(url)
  soup = BeautifulSoup(req.text)

  try:
    #appHubAppName
    title = soup.select_one('#appHubAppName').text.strip()

    '''
    추후에 이부분의 다른 게임들 크롤링을 다시 해야 할 듯 함
    '''
    #game_highlights > div.rightcol > div > div.glance_details
    #game_highlights > div.rightcol > div > div.game_description_snippet
    content = soup.select_one('#game_highlights > div.rightcol > div > div.game_description_snippet').text.strip()

    Image_url = soup.select_one('#gameHeaderImageCtn > img')['src']
    genres = soup.select_one('#glanceCtnResponsiveRight > div.glance_tags_ctn.popular_tags_ctn > div.glance_tags.popular_tags').text

    df.append(
        {'game_id' : game_id,
        'title' : title,
        'content' : content,
        'Image_url' : Image_url,
         'genres' : genres
        }
    )

  except:
    print(url)
    except_df.append(
        {'game_id' : game_id,
         'url' : url}
    )

https://store.steampowered.com/app/508150
https://store.steampowered.com/app/624910
https://store.steampowered.com/app/310380
https://store.steampowered.com/app/369200
https://store.steampowered.com/app/200110
https://store.steampowered.com/app/104700
https://store.steampowered.com/app/271290
https://store.steampowered.com/app/1193880
https://store.steampowered.com/app/1183970
https://store.steampowered.com/app/555440
https://store.steampowered.com/app/1056700
https://store.steampowered.com/app/942290
https://store.steampowered.com/app/974100
https://store.steampowered.com/app/597110
https://store.steampowered.com/app/802970
https://store.steampowered.com/app/854560
https://store.steampowered.com/app/783600
https://store.steampowered.com/app/569410
https://store.steampowered.com/app/542330
https://store.steampowered.com/app/751390
https://store.steampowered.com/app/560380
https://store.steampowered.com/app/285050
https://store.steampowered.com/app/336790
https://store.steampowered.com/

In [ ]:
df = pd.DataFrame(df)
df.to_csv(DATA_DIR + 'Steam_image_meta_data_v1_0624.csv', index=False)
df.head()

,game_id,title,content,Image_url,genres
0,768520,Red Solstice 2: Survivors,Sequel to the best-seller ​The Red Solstice​. ...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tStrategy RPG\t\t...
1,251570,7 Days to Die,7 Days to Die is an open-world game that is a ...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tEarly Access\t\t...
2,671510,DESOLATE,DESOLATE is a first person horror survival exp...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tSurvival\t\t\t\t...
3,613590,Dungeon Of Zaar - Open Beta,Dungeon of Zaar is a competitive tactical game...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tStrategy\t\t\t\t...
4,374280,Hired Ops,Hired Ops is a session-based online shooter de...,https://cdn.akamai.steamstatic.com/steam/apps/...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tFPS\t\t\t\t\t\t\...


In [ ]:
except_df = pd.DataFrame(except_df)
except_df.to_csv(DATA_DIR + 'Steam_except_data_v1_0624.csv', index=False)
except_df.head()

,game_id,url
0,508150,https://store.steampowered.com/app/508150
1,624910,https://store.steampowered.com/app/624910
2,310380,https://store.steampowered.com/app/310380
3,369200,https://store.steampowered.com/app/369200
4,200110,https://store.steampowered.com/app/200110


# 예외 데이터 크롤링1

In [ ]:
df = pd.read_csv(DATA_DIR + 'Steam_image_meta_data_v1_0624.csv')
except_df = pd.read_csv(DATA_DIR + 'Steam_except_data_v1_0624.csv')

In [ ]:
ret = []
except_ret = []
game_id_li = except_df['game_id'].unique()

for game_id in tqdm_notebook(game_id_li):

  url = f'https://store.steampowered.com/app/{game_id}'
  req = requests.get(url)

  try:
    if req.url == 'https://store.steampowered.com/':
      url = f'https://steamcommunity.com/app/{game_id}'
      req = requests.get(url)
      soup = BeautifulSoup(req.text)

      title = soup.select_one('#ModalContentContainer > div.apphub_background > div.apphub_HomeHeader > div.apphub_HomeHeaderContent > div.apphub_HeaderTop > div.apphub_AppName.ellipsis').text.strip()

      ret.append(
        {'game_id' : game_id,
        'title' : title,
        'content' : np.nan,
        'Image_url' : np.nan,
        'genres' : np.nan
        }
      )

    else:
      soup = BeautifulSoup(req.text)

      #appHubAppName
      title = soup.select_one('#appHubAppName').text.strip()
      #game_highlights > div.rightcol > div > div.game_description_snippet
      content = soup.select_one('#game_highlights > div.rightcol > div > div.glance_details').text.strip()
      #gameHeaderImageCtn > img
      Image_url = soup.select_one('#gameHeaderImageCtn > img')['src']
      #glanceCtnResponsiveRight > div.glance_tags_ctn.popular_tags_ctn > div.glance_tags.popular_tags
      genres = soup.select_one('#glanceCtnResponsiveRight > div.glance_tags_ctn.popular_tags_ctn > div.glance_tags.popular_tags').text

      ret.append(
          {'game_id' : game_id,
          'title' : title,
          'content' : content,
          'Image_url' : Image_url,
          'genres' : genres
          }
      )
  
  except:
    print(url)
    except_ret.append(
        {'game_id' : game_id,
         'url' : url}
    )

https://store.steampowered.com/app/33350
https://store.steampowered.com/app/48120
https://store.steampowered.com/app/345520
https://store.steampowered.com/app/39270
https://store.steampowered.com/app/98510
https://store.steampowered.com/app/227320
https://steamcommunity.com/app/213530
https://steamcommunity.com/app/330700
https://store.steampowered.com/app/431490
https://store.steampowered.com/app/1016870
https://store.steampowered.com/app/347890
https://store.steampowered.com/app/1158730
https://store.steampowered.com/app/3970
https://store.steampowered.com/app/433430
https://store.steampowered.com/app/641000
https://store.steampowered.com/app/35419
https://steamcommunity.com/app/753
https://steamcommunity.com/app/231410
https://store.steampowered.com/app/37700
https://steamcommunity.com/app/401980
https://steamcommunity.com/app/419270
https://store.steampowered.com/app/33340
https://steamcommunity.com/app/381570
https://steamcommunity.com/app/219820
https://steamcommunity.com/app/322

In [ ]:
ret_df = pd.DataFrame(ret)
ret_df.to_csv(DATA_DIR + 'Steam_image_meta_data_v2_0624.csv', index=False)
ret_df.head()

,game_id,title,content,Image_url,genres
0,508150,Primitive+ ARK Total Conversion,This content requires the base game ARK: Survi...,https://cdn.cloudflare.steamstatic.com/steam/a...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tAdventure\t\t\t\...
1,624910,3on3 FreeStyle,NaN,NaN,NaN
2,310380,Fractured Space,NaN,NaN,NaN
3,369200,Ghost in the Shell: Stand Alone Complex - Firs...,NaN,NaN,NaN
4,200110,Nosgoth,NaN,NaN,NaN


In [ ]:
except_ret_df = pd.DataFrame(except_ret)
except_ret_df.to_csv(DATA_DIR + 'Steam_except_data_v2_0624.csv', index=False)
except_ret_df.head()

,game_id,url
0,33350,https://store.steampowered.com/app/33350
1,48120,https://store.steampowered.com/app/48120
2,345520,https://store.steampowered.com/app/345520
3,39270,https://store.steampowered.com/app/39270
4,98510,https://store.steampowered.com/app/98510


# 예외 데이터 크롤링2

In [ ]:
except_ret_df = pd.read_csv(DATA_DIR + 'Steam_except_data_v2_0624.csv')

In [ ]:
df = []
game_id_li = except_ret_df['game_id'].unique()

for game_id in tqdm_notebook(game_id_li):

  url = f'https://store.steampowered.com/app/{game_id}'
  req = requests.get(url)
  soup = BeautifulSoup(req.text)

  try:
    try:
      title = soup.select_one('#appHubAppName').text.strip()

      content = soup.select_one('#game_highlights > div.rightcol > div > div.game_description_snippet').text.strip()

      Image_url = soup.select_one('#gameHeaderImageCtn > img')['src']
      genres = soup.select_one('#glanceCtnResponsiveRight > div.glance_tags_ctn.popular_tags_ctn > div.glance_tags.popular_tags').text

      df.append(
          {'game_id' : game_id,
          'title' : title,
          'content' : content,
          'Image_url' : Image_url,
          'genres' : genres
          }
      )

    except:
      # 장르 존재 X

      title = soup.select_one('#appHubAppName').text.strip()

      content = soup.select_one('#game_highlights > div.rightcol > div > div.glance_details').text.strip()

      Image_url = soup.select_one('#gameHeaderImageCtn > img')['src']

      genres = soup.select_one('#glanceCtnResponsiveRight > div > div.glance_tags.your_tags').text

      df.append(
          {'game_id' : game_id,
          'title' : title,
          'content' : content,
          'Image_url' : Image_url,
          'genres' : genres
          }
      )


  except:
    # 게임 존재 X
    df.append(
        {'game_id' : game_id,
          'title' : np.nan,
          'content' : np.nan,
          'Image_url' : np.nan,
          'genres' : np.nan}
    )

In [ ]:
df = pd.DataFrame(df)
df.to_csv(DATA_DIR + 'Steam_image_meta_data_v3_0624.csv', index=False)
df.head()

,game_id,title,content,Image_url,genres
0,33350,NaN,NaN,NaN,NaN
1,48120,The Settlers 7: Paths to a Kingdom,Choose your own path to victory in The Settler...,https://cdn.cloudflare.steamstatic.com/steam/a...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tStrategy\t\t\t\t...
2,345520,Infinite Crisis™,A sudden assault threatens the DC Multiverse. ...,https://cdn.cloudflare.steamstatic.com/steam/a...,\n\r\n\t\t\t\t\t\t\t\t\t\t\t\tFree to Play\t\t...
3,39270,NaN,NaN,NaN,NaN
4,98510,NaN,NaN,NaN,NaN
